# TEAM-SASA     
### Initial POC and EDA for ABS-CBN Website

In [48]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

## HELPER CLASS - WEBSCRAPING

In [49]:
DRIVER_PATH = 'C:\Program Files\chromedriver.exe'

class AbscbnScraper:

    def __init__(self, search_url):
        self.urls = []
        self.search_url = search_url
        self.driver = webdriver.Chrome(DRIVER_PATH)

    def crawl(self):
        self.driver.get(self.search_url)
        totalpages = self.get_total_pages()
        self.get_pages(totalpages)

    def get_driver(self):
        return self.driver
        
    def get_urls(self):
        return self.urls

    def get_urls_count(self):
        return len(self.urls)

    def get_total_pages(self):
        pages = self.driver.find_elements(By.CSS_SELECTOR, 'div.gsc-cursor-page')
        return len(pages)

    def add_urls(self, page):
        print(f'Start crawling article URLs for page {page}...')
        alinks = self.driver.find_elements(By.CSS_SELECTOR, 'a.gs-title')
        for a in alinks:
            link = a.get_attribute('href')
            if (link != None) and (link not in self.urls):
                self.urls.append(link)
        print('Done.')

    def get_pages(self, totalpages):
        print(f'Total number of pages to crawl: {totalpages}', end='\n\n')
        # Page 1
        self.add_urls(1)
        # Page 2 onwards
        self.driver.maximize_window()
        for p in range(1, totalpages):
            '''
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight)', '') 
            pages = self.driver.find_elements(By.CSS_SELECTOR, 'div.gsc-cursor-page')
            pages[p].click()
            '''
            url = "https://news.abs-cbn.com/special-pages/search?q=vaccination#gsc.tab=0&gsc.q=vaccination&gsc.page=%s" % (p+1)
            self.driver.get(url)
            sleep(5)
            self.add_urls(p+1)

    def close_popups(self):
        try:
            self.driver.find_element(By.XPATH, '//*[@id="survicate-box"]/div/div/div[3]/div[2]/button').click()
        except:
            pass
        try:
            self.driver.find_element(By.XPATH, '//*[@id="onesignal-slidedown-cancel-button"]').click()
        except:
            pass
        try:
            # Close footer ads
            self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]//following::div/p/span').click()
        except:
            pass

    def scrape_data(self, url):
        ### PREPARE ARTICLE URL
        self.driver.get(url)
        sleep(15)

        ### PREPARE TITLE
        title = self.driver.find_element(By.CSS_SELECTOR, 'h1.news-title')
        title = title.text
        print(f'\n[Title]\n{title}')

        ### PREPARE PUBLISHED DATE
        pubdate = self.driver.find_element(By.CSS_SELECTOR, 'p.timestamp-entry')
        pubdate = pubdate.text
        print(f'\n[Date Pulished]\n{pubdate}')
        

        ### PREPARE AUTHOR NAME
        author = self.driver.find_element(By.CSS_SELECTOR, 'h5.author-details')
        author = author.text
        print(f'\n[Author]\n{author}')

        # Try closing popups before clicking 'SEE THE RESULTS' button
        sleep(2)
        self.close_popups()

        ### PREPARE ARTICLE CONTENT
        article = self.driver.find_element(By.CSS_SELECTOR, 'div.article-content')
        article_content = article.text.replace('ADVERTISEMENT','').replace('How did this story make you feel?', '').replace('INSPIRED','')\
            .replace('HAPPY','').replace('AMUSED','').replace('NEUTRAL','').replace('ANNOYED','').replace('SAD','').replace('ANGRY','')\
            .replace('AFRAID','').replace('SEE THE RESULTS','').replace(' \n','').replace('\n ','').replace('\n\n','').replace('  ','') 
        print(f'\n[Content]\n{article_content}')
        '''
        ### PREPARE SENTIMENT RESULTS
        # Scroll to 'SEE THE RESULTS' button then simulate click to get sentiment results
        sleep(3)
        sentiment_classname = self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]/div[6]/div[4]/following::div').get_attribute('class')
        self.driver.execute_script(f'document.getElementsByClassName("{sentiment_classname}").scrollIntoView', '')
        
        # Automate click event to get sentiment results
        sentiment_index = 0
        for i in range(100):
            try:
                btn_sentiment = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{i}]/div/div/span')
                if btn_sentiment.text.upper() == 'SEE THE RESULTS':
                    sleep(2)
                    self.close_popups()
                    action = ActionChains(self.driver)
                    action.move_to_element(btn_sentiment).click().perform()
                    # btn_sentiment.click()
                    sentiment_index = i
                    break
            except:
                pass

        sleep(3)
        # Clean some sentiment texts
        sentiment_results = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{sentiment_index}]/div')
        sentiment_results = sentiment_results.text.replace('HOW OTHERS ARE FEELING', '').replace(' \n','').replace('\n ','').replace(' ARE ','-')
        # Convert each line of sentiment text to a list item
        sentiment_results = list(filter(None, sentiment_results.split('\n')))
        # Make a key-value pair of sentiment text and store it as dictionary object
        sentiment_results = {s.split('-')[1] : float(s.split('-')[0].split('%')[0])*.01 for s in sentiment_results}
        # Add a 0% value for sentiment key that does not have results in the current article and store as 'other sentiments'
        sentiment_keys = ['INSPIRED', 'HAPPY', 'AMUSED', 'NEUTRAL', 'ANNOYED', 'SAD', 'ANGRY', 'AFRAID']
        sentiment_others = {k : 0. for k in sentiment_keys if k not in sentiment_results}
        # Combine 'other sentiment dictionary' to the 'sentiment result dictionary' 
        sentiment_results = {**sentiment_results, **sentiment_others}
        print(f'\n[Sentiment]')
        print(sentiment_results)
        '''

        ### GET SENTIMENT RESULTS FROM ARTICLE
        return {'urls':url, 'titles':title, 'authors':author, 'published_dates':pubdate, 'articles_content':article_content}
    

# RUN WEB SCRAPING

In [50]:
KEYWORD = 'vaccine'
SEARCH_URL = f'https://news.abs-cbn.com/special-pages/search?q=vaccine%2BCOVID%2BPhilippines#gsc.tab=0&gsc.q={KEYWORD}&gsc.sort=date&gsc.page=1'

scraper = AbscbnScraper(SEARCH_URL)
scraper.crawl()

dataset = []
print(f'\nTotal number of articles: {scraper.get_urls_count()}')
for i,url in enumerate(scraper.get_urls()):
    try:
        print(f'\n\n\n[{i+1}]{" ":>5}SCRAPING DATA  ~>  {url}')
        data = scraper.scrape_data(url)
        dataset.append(data)
    except Exception as e:
        print(f'SCRAPING ERROR: {e}')
        pass

scraper.get_driver().quit()

Total number of pages to crawl: 10

Start crawling article URLs for page 1...
Done.
Start crawling article URLs for page 2...
Done.
Start crawling article URLs for page 3...
Done.
Start crawling article URLs for page 4...
Done.
Start crawling article URLs for page 5...
Done.
Start crawling article URLs for page 6...
Done.
Start crawling article URLs for page 7...
Done.
Start crawling article URLs for page 8...
Done.
Start crawling article URLs for page 9...
Done.
Start crawling article URLs for page 10...
Done.

Total number of articles: 99



[1]     SCRAPING DATA  ~>  https://news.abs-cbn.com/news/03/05/21/pagprayoridad-sa-senior-health-workers-sa-pagturok-ng-astrazeneca-vaccines-hinimok

[Title]
Pagprayoridad sa senior health workers sa pagturok ng AstraZeneca vaccines hinimok

[Date Pulished]
Posted at Mar 05 2021 05:41 PM | Updated as of Mar 05 2021 06:26 PM

[Author]
ABS-CBN News

[Content]
Ang kargamentong may laman na 487,200 doses ng AstraZeneca vaccines na ipinangako ng COVAX


[Title]
Ospital sa Iloilo unang nakatanggap ng Sinovac vaccine sa Western Visayas

[Date Pulished]
Posted at Mar 05 2021 10:12 PM

[Author]
Rolen Escaniel

[Content]
Photo courtesy of St. Paul Hospital-Iloilo
Unang nakatanggap ng Sinovac vaccine sa Western Visayas Biyernes ang St. Paul Hospital sa Iloilo.
Tinanggap nito ang unang 400 doses ng bakuna sa 1,500 doses na inilaan ng Department of Health Region 6 Biyernes ng umaga.COVID-19 vaccine na gawa ng AstraZeneca, dumating na sa Pilipinas
Unang nabakunahan sa hospital ang 52-anyos na si Dr. Maria Sylvia Teresa de Pili, chairman ng Internal Medicine Department ng hospital.
Si De Pili rin ang unang nabakunahan ng Sinovac sa isla ng Panay.
Bukod sa kaniya may 19 pang kawani ng ospital ang binakunahan.
Photo courtesy of St. Paul Hospital-Iloilo
RELATED VIDEO
Watch more in iWantTFC
Bisitahin ang Patrol.PH para sa iba pang mga balita.



[6]     SCRAPING DATA  ~>  https://news.abs-cbn.com/news/03/05/21/regional-hospitals-receive-sinovac-ja


[Title]
Duterte nagpasalamat sa COVAX sa pagdating ng AstraZeneca vaccines

[Date Pulished]
Posted at Mar 05 2021 07:46 PM | Updated as of Mar 05 2021 07:52 PM

[Author]
ABS-CBN News
SCRAPING ERROR: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.article-content"}
  (Session info: chrome=89.0.4389.72)




[10]     SCRAPING DATA  ~>  https://news.abs-cbn.com/business/03/05/21/first-batch-of-astrazeneca-covid-19-vaccine-for-private-sector-arriving-in-q2-concepcion

[Title]
First batch of AstraZeneca COVID-19 vaccine for private sector arriving in Q2: Concepcion

[Date Pulished]
Posted at Mar 05 2021 06:13 PM | Updated as of Mar 05 2021 06:25 PM

[Author]
ABS-CBN News

[Content]
Vials with a sticker reading "COVID-19 / Coronavirus vaccine / Injection only" and a medical syringe are seen in front of a displayed AstraZeneca logo in this illustration taken Oct. 31, 2020. Dado Ruvic, Reuters/File
MANILA - The initial batch of around 2.6 million do


[Title]
Vaccine rollout boosts private hospitals workers' confidence in Sinovac jabs

[Date Pulished]
Posted at Mar 05 2021 08:20 AM | Updated as of Mar 05 2021 08:27 AM

[Author]
ABS-CBN News

[Content]
Watch more in iWantTFC
MANILA - The country's coronavirus inoculation drive has bolstered the confidence of private hospital workers in receiving Sinovac jabs, an official said Friday.
Dr. Jose Rene De Grano, president of the Private Hospitals Association of the Philippines Inc, said many employees in private hospitals initially expressed reservation about the Chinese-made vaccines.
At least 70 percent of employees in private hospitals earlier agreed to be vaccinated against the disease.
"No'ng bigla pong sabihin na Sinovac, medyo bumaba. Pero pagkatapos po ng rollout sa PGH (Philippine General Hospital), sa mga different government hospital sa Metro Manila, medyo dumami na rin po ulit ang pumapayag. Sinasabi nila, kung ano man ang ibigay na bakuna ay tatanggapin na rin namin," he tol


[Title]
FAST FACTS: What you need to know about the AstraZeneca vaccine

[Date Pulished]
Posted at Mar 04 2021 08:25 PM

[Author]
ABS-CBN News

[Content]
Nearly a half million doses of AstraZeneca’s vaccines arrived in the country Thursday night, making it the 2nd COVID-19 vaccine brand to be distributed domestically. Palace photo
MANILA — More than 400,000 doses of AstraZeneca’s vaccines arrived in the Philippines on Thursday evening, making it the 2nd COVID-19 vaccine brand to be distributed in the country. Here’s what you need to know about AstraZeneca’s vaccine:
EFFICACY
The COVID-19 Vaccine (ChAdOx1-S[recombinant]) or AZD1222 vaccine was developed by the British pharmaceutical company AstraZeneca and the Oxford University.
It has an efficacy rate of 63.09% against symptomatic COVID-19 infection, according to the World Health Organization, which included AstraZeneca in its emergency use listing.
The Philippine Food and Drug Administration said it has an overall efficacy rate of 70

...


[Title]
PH begins COVID-19 vaccinations

[Date Pulished]
Posted at Mar 01 2021 10:48 PM

[Author]
ABS-CBN News
SCRAPING ERROR: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.article-content"}
  (Session info: chrome=89.0.4389.72)




[86]     SCRAPING DATA  ~>  https://news.abs-cbn.com/news/03/01/21/from-the-bottom-of-my-heart-duterte-thanks-chinas-xi-for-sinovac-donation

[Title]
'From the bottom of my heart': Duterte thanks China's Xi for Sinovac donation

[Date Pulished]
Posted at Mar 01 2021 10:33 PM

[Author]
Rose Carmelle Lacuata, ABS-CBN News

[Content]
MANILA - President Rodrigo Duterte on Monday thanked Chinese President Xi Jinping for helping the Philippines get its first batch of COVID-19 vaccines from Sinovac Biotech.
"If President Xi Jinping is listening now, President, from the bottom of my heart, and with immense gratitude, I thank you, the Chinese people and government, for being so generous. Maybe someday I can repay you i


[Title]
FAST FACTS: The COVID-19 vaccination rollout in the Philippines

[Date Pulished]
Posted at Mar 01 2021 07:44 PM

[Author]
ABS-CBN News

[Content]
Nurse Chareluck Santos receives her first dose of the Sinovac vaccine at the Philippine General Hospital this March 1, 2021. She was the health worker in charge of vaccinating Dr. Gerardo Legaspi, the hospital's director and first recipient of the vaccine.
MANILA — The Philippines has started its vaccination program with jabs from Beijing-based drug maker Sinovac Biotech on Monday, in hopes of an immediate return to normalcy as the country races to stop the spread of the coronavirus disease (COVID-19).This year, the country aims to inoculate 70 million of its 108 million people to achieve herd immunity and reopen an economy that in 2020 saw its worst contraction on record, due largely to tight restrictions on movement in place since mid-March.
Here's what you need to know about the vaccination program in the Philippines:
What vaccine


[Title]
Lung Center of the Philippines handa na para sa COVID-19 vaccination

[Date Pulished]
Posted at Feb 28 2021 01:37 PM

[Author]
ABS-CBN News

[Content]
Lung Center of the Philippines sa Quezon City. George Calvelo, ABS-CBN News/File
MAYNILA – Kasado na ang nakatakdang pagsisimula umaga ng Lunes ng COVID-19 vaccination sa Lung Center of the Philippines sa Quezon City.
Isa ang Lung Center sa mga referral hospitals para sa mga pasyenteng may COVID-19.
Sa advisory ng ospital, nakasaad na magsisimula ang mga aktibidad para sa vaccination alas-9:30 ng umaga. Pangungunahan umano ito nina Health Secretary Francisco Duque III at Metropolitan Manila Development Authority (MMDA) Chairman Benhur Abalos Jr.
Sa naturang aktibidad din tatanggapin ni Duque ang bakuna ng Sinovac, na inaasahang dadating galing China ngayong Linggo.
Bukod kay Duque, babakunahan din ang 20 inisyal na staff ng ospital para sa photo session.
EXPLAINER: What's behind varying efficacy data for Sinovac's COVID-19 vacci


[Title]
Thailand starts COVID-19 vaccination campaign

[Date Pulished]
Posted at Feb 28 2021 12:46 PM

[Author]
Reuters

[Content]
A nurse is injected with the CoronaVac vaccine, developed by China’s Sinovac firm, as the first batch of vaccines to battle the COVID-19 coronavirus are administered to frontline health workers at the Bamrasnaradura Infectious Diseases Institute in Bangkok on February 28, 2021. Lillian Suwanrumpha, Agence France-Presse
BANGKOK - Thailand kicked off its COVID-19 inoculation campaign on Sunday, with cabinet ministers, health officials and medical professionals among the first in the queue to receive vaccinations.
The first doses of vaccine, developed by China's Sinovac Biotech, were given to Deputy Prime Minister Anutin Charnvirakul, who is also the health minister, among others at an infectious diseases institute on the outskirts of Bangkok.
"I hope that the vaccination will result in people being safe from the spread of COVID-19 and it allows Thailand to r


[Title]
Chinese teams looking for vaccines for COVID-19 variants for ‘a long time’, expert says

[Date Pulished]
Posted at Feb 28 2021 01:47 PM

[Author]
Mimi Lau, South China Morning Post

[Content]
Chinese teams began looking at vaccines to fight Covid-19 variants “a long time ago”, according to a leading military medical scientist.
“We are running extended targeted studies tracking [vaccine] safety via national databases. We are also tracking the vaccine’s efficacy rate against the latest mutations,” Chen Wei, a researcher at the Institute of Military Medicine under the Academy of Military Sciences, said in an interview with state broadcaster CCTV on Saturday.
“We have been tracking and analysing a global database of Covid-19 patients with more than 400,000 samples through genetics and biologics. Our labs then cross-examine the responses of virus variants – such as the Brazilian, South African and British strains – to existing vaccines,” she said.
Chen, who is also a member of the 

# EDA

In [51]:
df = pd.DataFrame(dataset)
df

,urls,titles,authors,published_dates,articles_content
0,https://news.abs-cbn.com/news/03/05/21/pagpray...,Pagprayoridad sa senior health workers sa pagt...,ABS-CBN News,Posted at Mar 05 2021 05:41 PM | Updated as of...,"Ang kargamentong may laman na 487,200 doses ng..."
1,https://news.abs-cbn.com/business/03/05/21/phi...,"Philippine Airlines, Cebu Pacific help transpo...",ABS-CBN News,Posted at Mar 05 2021 04:56 PM,"Jacque Manabat, ABS-CBN News; Ciele Villaluna,..."
2,https://news.abs-cbn.com/overseas/03/05/21/sin...,Sinovac's COVID-19 vaccine may not trigger suf...,Reuters,Posted at Mar 05 2021 05:34 PM,Health workers from Sta. Ana Hospital administ...
3,https://news.abs-cbn.com/overseas/03/05/21/cub...,Cuban-developed vaccine enters Phase III trial,Agence France-Presse,Posted at Mar 05 2021 11:15 AM,Cuba's Finlay vaccine institute has announced ...
4,https://news.abs-cbn.com/news/03/05/21/ospital...,Ospital sa Iloilo unang nakatanggap ng Sinovac...,Rolen Escaniel,Posted at Mar 05 2021 10:12 PM,Photo courtesy of St. Paul Hospital-Iloilo\nUn...
...,...,...,...,...,...
74,https://news.abs-cbn.com/overseas/03/01/21/chi...,China's CanSino says first vaccines packaged i...,Reuters,Posted at Mar 01 2021 06:39 AM,A towing tractor truck pulls a container trans...
75,https://news.abs-cbn.com/overseas/02/28/21/tha...,Thailand starts COVID-19 vaccination campaign,Reuters,Posted at Feb 28 2021 12:46 PM,A nurse is injected with the CoronaVac vaccine...
76,https://news.abs-cbn.com/news/02/28/21/health-...,Health workers can wait for other COVID-19 vac...,"John Gabriel Agcaoili, ABS-CBN News",Posted at Feb 28 2021 09:30 PM,The Medical City in Pasig City simulates the f...
77,https://news.abs-cbn.com/news/03/01/21/set-you...,"'Set your fears aside,' Duterte tells Filipino...","Jamaine Punzalan, ABS-CBN News",Posted at Mar 01 2021 12:08 AM,MANILA — President Rodrigo Duterte on Sunday u...


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   urls              79 non-null     object
 1   titles            79 non-null     object
 2   authors           79 non-null     object
 3   published_dates   79 non-null     object
 4   articles_content  79 non-null     object
dtypes: object(5)
memory usage: 3.2+ KB


In [53]:
df.describe().T

,count,unique,top,freq
urls,79,79,https://news.abs-cbn.com/spotlight/03/05/21/ho...,1
titles,79,79,Nine great apes in San Diego Zoo vaccinated fo...,1
authors,79,27,ABS-CBN News,35
published_dates,79,79,Posted at Mar 01 2021 09:55 AM | Updated as of...,1
articles_content,79,79,MANILA - The Philippine General Hospital (PGH)...,1


In [54]:
df.isna().sum()

urls                0
titles              0
authors             0
published_dates     0
articles_content    0
dtype: int64

In [64]:
### Save a CSV dataset
df.to_csv(f'abscbn_{KEYWORD}.csv', index=False)

# WORDCLOUD

In [72]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np

ImportError: DLL load failed while importing query_integral_image: The specified module could not be found.

In [68]:
import sys
print(sys.executable)


C:\Users\gear\AppData\Local\Programs\Python\Python39\python.exe


In [ ]:
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % np.random.randint(60, 100)

mask = np.array(Image.open('bg.png'))

words = ' '.join(df['articles_content'].tolist())
wordcloud = WordCloud(background_color='black', max_words=2000, width=1024, height=720, mask=mask)
wordcloud.generate(words)

plt.figure(figsize=(20, 8), dpi=300)
plt.title('Rappler on COVID Vaccine in PH (Word Cloud)')
plt.imshow(wordcloud.recolor(color_func=grey_color_func, random_state=42), interpolation='bilinear')
plt.axis('off')
plt.show()